In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

# Read the data

In [3]:
dfXtrain = pd.read_csv('preprocessed_csv/train_more.csv', index_col='id')
dfXtest = pd.read_csv('preprocessed_csv/test_more.csv', index_col='id')
dfYtrain = pd.read_csv('preprocessed_csv/y_train_more.csv', header=None, names=['INDEX', 'P_TARGET_FLAG'])

In [85]:
x_train = np.array(dfXtrain)
x_test = np.array(dfXtest)

y_train = np.array(dfYtrain['P_TARGET_FLAG'])

# Save routines

In [5]:
dfYtest = pd.DataFrame({'INDEX': dfXtest.index, 'P_TARGET_FLAG': np.zeros(x_test.shape[0])})
dfYtest.head()

,INDEX,P_TARGET_FLAG
0,3,0.0
1,9,0.0
2,10,0.0
3,18,0.0
4,21,0.0


In [6]:
def save_to_file(y, file_name):
    dfYtest['P_TARGET_FLAG'] = y
    dfYtest.to_csv('results/{}'.format(file_name), index=False)

# Train XGB

In [9]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split

In [86]:
x_subtrain, x_subtest, y_subtrain, y_subtest = train_test_split(x_train, y_train, 
                                                                test_size=0.15, random_state=42,
                                                                stratify=y_train)

In [16]:
XGBC = xgb.XGBClassifier

In [87]:
kwargs = {'n_estimators': 300, 'objective':'binary:logistic', 'missing': -1, 'seed': 56,
          'reg_alpha': 0.1}

clf = XGBC(**kwargs)
clf.fit(x_subtrain, y_subtrain, eval_metric='auc')
proba = clf.predict_proba(x_subtest)[:, 1]

score = roc_auc_score(y_subtest, proba)
score

0.82585310936137779

In [88]:
kwargs = {'n_estimators': 300, 'objective':'binary:logistic', 'missing': -1, 'seed': 56,
          'reg_alpha': 0.1}

clf = XGBC(**kwargs)
clf.fit(x_train, y_train, eval_metric='auc')
proba = clf.predict_proba(x_test)[:, 1]

save_to_file(proba, 'xbgoost.csv')